# Using SimStore

This notebook will show the current way to use SimStore if you're starting a new project. This will change over time as more functionality gets added to SimStore.

We'll load an existing trajectory using a netcdfplus file. Other than that, all objects will be created fresh and in the most OPS-2.0 style currently possible.

NOTE: The main caveat here is that you cannot disk-cache results of an old CV with new storage. However, once new-style CVs are available, you can disk-cache a new CV in new storage.

In [1]:
import openpathsampling as paths
from openpathsampling.engines import toy as toys

## Load up old sample set



In [2]:
old_storage = paths.Storage("../toy_mstis_1k_OPS1_py36.nc")
samples = old_storage.steps[-1].active

## Create simulation objects

This is all taken from the standard OPS MSTIS example.

In [3]:
pes = (toys.OuterWalls([1.0, 1.0], [0.0, 0.0])
       + toys.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4])
       + toys.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5])
       + toys.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5]))

topology=toys.Topology(n_spatial=2,
                       masses=[1.0, 1.0],
                       pes=pes)

integ = toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={'integ': integ,
         'n_frames_max': 5000,
         'n_steps_per_frame': 1}

engine = toys.Engine(options=options,
                      topology=topology).named('toy_engine')

# note: no need for a template!

In [4]:
def circle(snapshot, center):
    import math
    return math.sqrt((snapshot.xyz[0][0]-center[0])**2
                     + (snapshot.xyz[0][1]-center[1])**2)
    
# in the future, this will use a CV class based on SimStore
opA = paths.CoordinateFunctionCV(name="opA", f=circle, center=[-0.5, -0.5])
opB = paths.CoordinateFunctionCV(name="opB", f=circle, center=[0.5, -0.5])
opC = paths.CoordinateFunctionCV(name="opC", f=circle, center=[0.0, 0.4])

In [5]:
stateA = paths.CVDefinedVolume(opA, 0.0, 0.2).named("A")
stateB = paths.CVDefinedVolume(opB, 0.0, 0.2).named("B")
stateC = paths.CVDefinedVolume(opC, 0.0, 0.2).named("C")

interfacesA = paths.VolumeInterfaceSet(opA, 0.0, [0.2, 0.3, 0.4])
interfacesB = paths.VolumeInterfaceSet(opB, 0.0, [0.2, 0.3, 0.4])
interfacesC = paths.VolumeInterfaceSet(opC, 0.0, [0.2, 0.3, 0.4])

In [6]:
ms_outers = paths.MSOuterTISInterface.from_lambdas(
    {ifaces: 0.5
     for ifaces in [interfacesA, interfacesB, interfacesC]}
)
mstis = paths.MSTISNetwork(
    [(stateA, interfacesA),
     (stateB, interfacesB),
     (stateC, interfacesC)],
    ms_outers=ms_outers
)

In [7]:
%%time
# this is slow because it has to load the trajectories from netcdfplus
scheme = paths.DefaultScheme(mstis, engine=engine)
init_conds = scheme.initial_conditions_from_trajectories(samples)

No missing ensembles.
No extra ensembles.
CPU times: user 15.3 s, sys: 1.26 s, total: 16.6 s
Wall time: 16.7 s


## Use new storage in a simulation

In [8]:
from openpathsampling.experimental.simstore import SQLStorageBackend
from openpathsampling.experimental.storage import Storage

In [9]:
# when you use old CVs, you need to monkey patch
# this cell will be removed once storable functions have been added
from openpathsampling.experimental.storage.monkey_patches import monkey_patch_saving
paths = monkey_patch_saving(paths)

In [10]:
backend = SQLStorageBackend("storage_01.db", mode='w')
storage = Storage.from_backend(backend)

In [11]:
simulation = paths.PathSampling(storage=storage,
                                move_scheme=scheme,
                                sample_set=init_conds)

In [12]:
simulation.save_frequency = 50  # with toy models, we can hold many steps in memory
simulation.run(500)

Working on Monte Carlo cycle number 500
Running for 27 seconds -  0.05 seconds per step
Estimated time remaining: 0 seconds
DONE! Completed 500 Monte Carlo cycles.


In [13]:
print(storage.summary())

File: storage_01.db
Includes tables:
* samples: 671 items
* sample_sets: 501 items
* trajectories: 393 items
* move_changes: 2334 items
* steps: 501 items
* details: 2305 items
* simulation_objects: 399 items
* snapshot0: 1505 items
* snapshot1: 12601 items



This finds two types of snapshots because the engine on the snapshots we loaded does not have the same UUID as the one we created here.

In [14]:
old_storage.close()
storage.close()
# new storage doesn't really need to be closed

In [15]:
%%time
# quick calculation of overall acceptance
sum([step.change.accepted for step in storage.steps])
# main performance issue here is that we need to load all the snapshots
# the fix for this is to make trajectories lazy-loading, but that breaks old
# CVs, so that has to wait for the new CVs

CPU times: user 3.86 s, sys: 103 ms, total: 3.96 s
Wall time: 4.06 s


341